# Bien choisir les ressources
Il y a plusieurs **ressources de calcul haute performance** qui vous
sont disponibles à l'Alliance de recherche numérique du Canada :
* Grappes de calcul :
  * Béluga, Cedar, Graham, Narval, Niagara
* Stockage :
  * Temporaire, projet, *nearline*, dépôt de données de recherche

## Objectif - construction d'un script de tâche Slurm
Le principal objectif de ce chapitre est de vous permettre d'analyser
vos besoins dans le but de **déterminer les ressources nécessaires**
pour vos tâches de calcul.
Chaque tâche de calcul est définie par un **script de tâche** destiné à
[l'ordonnanceur Slurm](https://slurm.schedmd.com/documentation.html).
Typiquement écrit en commandes Bash, on y retrouve :
* Le [shebang](https://fr.wikipedia.org/wiki/Shebang) en toute
  première ligne. Par exemple : `#!/bin/bash`
* Les options `#SBATCH` en entête pour les besoins de la tâche. Les
  options en entête seront lues par la commande de soumission de tâche
  [`sbatch`](https://slurm.schedmd.com/sbatch.html)
* [Chargement des modules](https://docs.alliancecan.ca/wiki/Utiliser_des_modules) requis
* Les commandes Bash qui seront exécutées automatiquement sur des
  processeurs réservés pour la tâche

Par exemple : [`scripts/mpi-allo.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/mpi-allo.sh)

```Bash
nano scripts/mpi-allo.sh  # Ctrl+Q pour quitter
```
```
#!/bin/bash
#SBATCH --ntasks=10
#SBATCH --mem-per-cpu=1000M
#SBATCH --time=0-00:10

module load gcc/9.3.0 &> /dev/null

mpirun printenv HOSTNAME OMPI_COMM_WORLD_RANK OMPI_COMM_WORLD_SIZE
```

Notre documentation à cet effet débute à la page :
[Exécuter des tâches](https://docs.alliancecan.ca/wiki/Running_jobs/fr)

## Analyse des calculs localement
Lorsqu'une tâche est en cours d'exécution sur votre ordinateur,
vous pouvez surveiller différentes métriques :
* Utilisation CPU (et GPU, s'il y a lieu)
* Mémoire-vive utilisée
* Accès au stockage

### Sous Windows
* [Gestionnaire des tâches Windows](https://fr.wikipedia.org/wiki/Gestionnaire_des_t%C3%A2ches_Windows)
* Pour le faire afficher, on le trouve de deux manières :
  * Chercher *Gestionnaire des tâches* dans le menu Démarrer
  * Raccourcis clavier Ctrl+Alt+Suppr

![Aperçu du gestionnaire des tâches Windows](images/win-task-manager.png)

### Sous Mac OS
* [Moniteur d’activité](https://support.apple.com/fr-ca/guide/activity-monitor/actmntr1001/mac)
* Pour le faire afficher :
  * Démarrer l'application à partir des *Applications et Utilitaires* de Mac OS
  * Sinon, utiliser le raccourcis clavier Commande+Espace et taper les
    premières lettres de "Moniteur d'activité" pour pouvoir sélectionner
    cette application

![Aperçu du moniteur d'activité Mac OS](https://help.apple.com/assets/5FDCF1894EB74318147EC0CF/5FDCF18A4EB74318147EC0D6/fr_CA/ad6337d66061aa27122e75521960fc5a.png)

### Sous Linux
Dans un terminal Unix, on peut utiliser :
* La [commande `top`](https://man7.org/linux/man-pages/man1/top.1.html) (`q` pour quitter)

![Capture de top](images/linux-top.png)

* La [commande `htop`](https://man7.org/linux/man-pages/man1/htop.1.html) (`q` pour quitter)

![Capture de htop](images/linux-htop.png)

## Analyse des tâches sur la grappe de calcul
On commence par **se connecter au noeud frontal** de la grappe :
```Bash
ssh login1
...
```
**Note 1** : pour accéder aux grappes de calcul en production,
il vaut mieux utiliser [une paire de clés SSH](https://docs.alliancecan.ca/wiki/Using_SSH_keys_in_Linux/fr).

**Note 2** : Avec votre accès par défaut, vous avez un compte de
calcul `def-*` de base qui vous permet de lancer des tâches.

Pour soumettre un script de tâche, on utilise la
[commande `sbatch`](https://slurm.schedmd.com/sbatch.html) :
```Bash
sbatch scripts/blastn-gen-seq.sh
```

Et pour voir l'état de la tâche, on utilise la
[commande `squeue`](https://slurm.schedmd.com/squeue.html) :
```Bash
squeue -u $USER  # ou 'sq'
```

### Ressources utilisées d'une tâche terminée
Avec la [commande `sacct`](https://slurm.schedmd.com/sacct.html),
on peut obtenir un tableau détaillé de nos tâches exécutées
depuis minuit :
```Bash
sacct
```

Avec la [commande `seff`](https://docs.alliancecan.ca/wiki/Running_jobs/fr#T.C3.A2ches_termin.C3.A9es),
on peut obtenir un court rapport d'exécution d'une tâche.
Ce rapport inclut une mesure du temps écoulé, une mesure du temps CPU
et une mesure de consommation maximale de la mémoire-vive.
Des valeurs d'efficacité en pourcentages sont données pour les cycles CPU
et la mémoire-vives en fonction des quantités réservées.
```Bash
seff <No_tâche>
```

### Ressources utilisées par une tâche CPU en cours
Étant donné un certain calcul matriciel dans le script Python 
[`scripts/crunch.py`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/crunch.py) :

```Bash
less scripts/crunch.py   # q pour quitter
```

Lors d'une tâche interactive, on peut utiliser `top` et `htop` pour surveiller les ressources utilisées :

```Bash
# Tâche interactive
salloc --ntasks-per-node=4 --mem=8000M --time=0:15:0

less scripts/crunch.sh  # q pour quitter

# Exécution avec un processeur
bash scripts/crunch.sh --cpu 1
top -u $USER  # q pour quitter

# Exécution avec 4 processeurs
bash scripts/crunch.sh --cpu 4
htop -u $USER  # q pour quitter

# Comparer les résultats
grep sec *.log

exit  # Pour revenir à login1
```

#### Exercice - Valider le comportement d'une tâche active
Pendant que vos tâches sont actives, vous pouvez vous connecter par
SSH aux noeuds de calcul correspondants afin de valider que
l'exécution se passe bien :
```Bash
cat scripts/inv-mat.sh
sbatch scripts/inv-mat.sh
```

Voici les étapes de validation (à adapter pour l'exercice) :
* Identification du ou des noeud(s) avec : `squeue -u $USER`
* Connexion avec : `ssh <nom_noeud>`
* Inspection avec `top` et/ou `htop` :
  * Est-ce que vos processus s'exécutent avec un **pourcentage de 100%?**
  * Est-ce que vos processus parallèles s'exécutent avec un
    **pourcentage de $n$ * 100%**, où $n$ est le nombre de processeurs
    par tâche Slurm?
  * Est-ce que le **noeud de calcul** semble pleinement utilisé?
* **Inspection des résultats**
  * Identifier tout problème, s'il y a lieu; trouver la cause
  * Corriger le code source, la compilation, le script ou les
    paramètres de la tâche de calcul
  * Relancer la tâche de calcul et refaire les précédentes étapes

### (Démo) Ressources utilisées par une tâche GPU en cours
```Bash
# Tâche interactive
salloc --ntasks-per-node=4 --mem=8000M --time=0:15:0 --gres=gpu:1
```

* Pour Windows et Mac OS, il existe des outils propriétaires
  permettant de visualiser en temps réel l'utilisation du GPU.
  Veuillez vous référer au site Web du manufacturier de votre GPU
  pour les détails
* Sous Linux, il y a d'abord la
  [commande `nvidia-smi`](https://developer.nvidia.com/nvidia-system-management-interface)

```Bash
nvidia-smi
```

![Capture nvidia-smi](images/nvidia-smi.png)

* Il existe aussi un projet [`nvtop`](https://github.com/Syllo/nvtop)
  permettant de visualiser l'utilisation d'un ou plusieurs GPUs dans
  un terminal :

```Bash
# Exécution avec un GPU
bash scripts/crunch.sh --gpu
nvtop  # q pour quitter
```
![Capture nvtop](https://raw.githubusercontent.com/Syllo/nvtop/master/screenshot/NVTOP_ex1.png)

```Bash
# Regarder le résultat
less tg.log    # q pour quitter

exit  # Pour revenir à login1
```

#### Exercice - Tester Crunch avec un GPU
```Bash
less   scripts/crunch-sbatch-1gpu.sh
sbatch scripts/crunch-sbatch-1gpu.sh

squeue -u $USER  # Pour vérifier l'état de la tâche
tail -24 $(ls slurm-* | tail -1)
```

#### Comparer la vitesse CPU vs GPU
Avant d'utiliser massivement les GPUs d'une grappe de calcul, il faut
tout d'abord que l'application ou l'algorithme puisse démontrer une
"bonne performance" en utilisant plusieurs processeurs en parallèle.

Quelques définitions :
* **Temps écoulé** = temps d'exécution total que l'on perçoit et non le temps CPU
* **Accélération** = (temps avec un processeur) / (temps avec parallélisme)
* **Efficacité** = (Accélération) / (nombre de processeurs)

Le coût d'un noeud GPU étant cinq fois supérieur à celui d'un noeud
régulier, l'utilisation d'un seul GPU doit permettre une accélération
d'au moins cinq fois (5x) la vitesse de huit (8) à douze (12) processeurs.
* **Accélération** = (temps avec 8 à 12 processeurs) / (temps avec un accélérateur)

## Estimer les ressources nécessaires
### Efficacité cible d'une tâche
À coût d'essais et erreurs avec une **tâche de petite taille**,
la cible pour :
* **Le calcul** est une **efficacité d'au moins 90%**
  * Tâches séquentielles : il faut **optimiser les accès aux données**
    * Utiliser adéquatement les différents types de stockage
  * Tâches parallèles : il existe un **nombre maximal de processeurs**
    à utiliser pour respecter ce seuil :
    * Voir la figure dans la page
      [loi d'Amdahl](https://fr.wikipedia.org/wiki/Loi_d%27Amdahl)
* **La mémoire-vive** est une consommation **de l'ordre de 80%**
  de ce qui est demandé à l'ordonnanceur Slurm

Rappel - vous pouvez obtenir ces pourcentages via les commandes `sacct -X` (surtout pour obtenir les numéros de tâches) et `seff`. Les valeurs à considérer sont :
* `CPU Utilized` et `CPU Efficiency`
* `Memory Utilized` et `Memory Efficiency`

#### Exercice - Efficacité des tâches
Pour quelques-unes de vos tâches listées par la commande suivante :
```Bash
sacct -X
```
Obtenez le `CPU Efficiency` et le `Memory Efficiency` via la commande :
```Bash
seff <No_tâche>
```

### La complexité des algorithmes
La question qui se pose :
**en augmentant la ou les dimensions du problème**,
quelles devraient être la durée du calcul et la consommation en mémoire-vive?
Une [analyse de la complexité de l'algorithme principal](https://fr.wikipedia.org/wiki/Analyse_de_la_complexit%C3%A9_des_algorithmes)
permettrait de connaître l'ordre $O$ du calcul en fonction de la taille $n$ des données :

* $O(n)$: proportionnel à $n$
* $O(n*m)$: représente un calcul à deux (2) dimensions indépendantes
* $O(n^3)$: calcul d'ordre cubique
* $O(n*m*k^2)$: par exemple, un filtre de taille $k*k$ sur une image $n*m$
* $O(n*log(n))$: typique de certains
[algorithmes de tri](https://fr.wikipedia.org/wiki/Algorithme_de_tri#Comparaison_des_algorithmes)
où il y a $n$ éléments à trier en $log_2(n)$ étapes.

Une analyse détaillée du code n'est pas nécessaire pour déterminer
le type de calcul qui est fait :
* Vous pouvez considérer le **format des données en entrées** pour
  deviner l'ordre du calcul principal. 
* Vous pouvez **mesurer le temps d'exécution** en fonction de la
  taille du problème. En extrapolant les résultats, il serait possible
  de prévoir le comportement du programme sur une grappe de calcul.

### Taille des données et nombre de fichiers à traiter
En plus du temps de calcul et de l'espace mémoire, il faut aussi
considérer l'utilisation du stockage. Les valeurs à tenir compte :
1. La **quantité** en octets (ou Go)
    * Peut servir à **estimer** l'utilisation de la mémoire-vive
    * Tenir compte de la taille du **stockage local rapide** pour
      optimiser les accès aux fichiers
1. Le **nombre** de fichiers à traiter
    * Considérer le **parallélisme de données**
    * **Multiplier la durée moyenne** du traitement d'un fichier par
      le nombre de fichiers pour estimer la durée d'une tâche
    * **Multiplier la taille moyenne** des fichiers par leur nombre
      pour estimer l'espace en mémoire-vive (par exemple : des images)
    * Utiliser le stockage rapide pour **optimiser les accès**
      aléatoires et nombreux

Pour obtenir le nombre de fichiers et la taille totale :
* **Sous Windows** : dans l'explorateur Windows (raccourcis clavier : Windows + E)
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Propriétés*

![Windows data properties](images/win-data-size.png)

* **Sous Mac OS** : dans *Finder*
  * Sélectionner un dossier ou plusieurs fichiers
  * Bouton droit de la souris -> *Get Info*
  * Autrement : avec l'affichage *Par liste*
    * [Activer *Calculer toutes les tailles*](https://www.solutionenligne.org/comment-afficher-taille-dossiers-fichiers-dans-finder-mac-os/)

* **Sous Linux** et sur les **grappes de calcul** :
  * L'environnement graphique peut offrir le même genre d'outils, mais tout dépend de la distribution et du bureau.
  * La commande `du -bs DOSSIER` (`b` : taille apparente en octets, `s` : somme totale) calcule récursivement et affiche la taille totale en octets. La taille apparente est celle qui importe lors d'un transfert ou d'une sauvegarde de données.
  * La commande `find DOSSIER | wc -l` compte récursivement et affiche le nombre de fichiers et de sous-dossiers.

```Bash
du -bs donnees
find donnees | wc -l
```

## Principales différences entre les grappes de calcul
* À propos des grappes :

| | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar/fr) | [Graham](https://docs.alliancecan.ca/wiki/Graham/fr) | [Narval](https://docs.alliancecan.ca/wiki/Narval) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara/fr) |
|-----------------------:|:---------:|:---------:|:---------:|:------------:|:----------:|
| **Mise en production** | Mars 2019 | Juin 2017 | Juin 2017 | Octobre 2021 | Avril 2018 |
|              **Ville** | Montréal  |  Burnaby  | Waterloo  |   Montréal   |   Toronto  |
|           **Province** |  Québec   |    C.-B.  |  Ontario  |    Québec    |   Ontario  |

* Nombre de processeurs (coeurs CPU) selon le cas :

| Processeur Intel/AMD  | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar/fr) | [Graham](https://docs.alliancecan.ca/wiki/Graham/fr) | [Narval](https://docs.alliancecan.ca/wiki/Narval) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara/fr) |
|----------------------:|:--------:|:--------:|:--------:|:---------:|:---------:|
|      Broadwell (avx2) |          | 724 * 32 | 983 * 32 |           |           |
|      Skylake (avx512) | 802 * 40 | 640 * 48 |          |           | 1548 * 40 |
| Cascade Lake (avx512) |          | 768 * 48 |  72 * 44 |           |  468 * 40 |
|      EPYC Rome (avx2) |          |          |          | 1181 * 64 |           |

| Mémoire par proc. | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar/fr) | [Graham](https://docs.alliancecan.ca/wiki/Graham/fr) | [Narval](https://docs.alliancecan.ca/wiki/Narval) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara/fr) |
|-------:|:-----:|:-----:|:-----:|:-----:|:-----:|
|  2400M |  6400 |       |       |       |       |
|  4000M |       | 86016 | 28896 | 73280 |       |
|  4400M |       |       |  3168 |       |       |
|  4800M | 23560 |       |       |       | 80960 |
|  8000M |       |  3072 |  1792 |       |       |
| 16000M |       |   768 |   768 |       |       |
| 19200M |  2120 |       |       |       |       |
| 32000M |       |       |       |  2304 |       |
| 48000M |       |   768 |   192 |       |       |
| 96000M |       |   128 |       |       |       |

* [Nombre de GPUs](https://docs.alliancecan.ca/wiki/Using_GPUs_with_Slurm/fr) selon le cas :

| Accélérateurs | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar/fr) | [Graham](https://docs.alliancecan.ca/wiki/Graham/fr) | [Mist (Power9)](https://docs.scinet.utoronto.ca/index.php/Mist) | [Narval](https://docs.alliancecan.ca/wiki/Narval) |
|----------------:|:---:|:---:|:---:|:---:|:---:|
| NVIDIA P100 12G |     | 456 | 320 |     |     |
| NVIDIA P100 16G |     | 128 |     |     |     |
|   NVIDIA T4 16G |     |     | 144 |     |     |
| NVIDIA V100 16G | 688 |     |  54 |     |     |
| NVIDIA V100 32G |     | 768 |  16 | 216 |     |
| NVIDIA A100 40G |     |     |     |     | 636 |

* Réseau haute-performance et ordonnancement :

| | [Béluga](https://docs.alliancecan.ca/wiki/B%C3%A9luga) | [Cedar](https://docs.alliancecan.ca/wiki/Cedar/fr) | [Graham](https://docs.alliancecan.ca/wiki/Graham/fr) | [Narval](https://docs.alliancecan.ca/wiki/Narval) | [Niagara](https://docs.alliancecan.ca/wiki/Niagara/fr) |
|-----------------------:|:----------:|:-----------:|:----------:|:-----------:|:----------:|
|       Connexion rapide | InfiniBand |   OmniPath  | InfiniBand | InfiniBand  | InfiniBand |
|              Topologie |  En arbre  |   En arbre  |  En arbre  |  En arbre   | DragonFly+ |
|    Taille îlots (proc) | 640 à 1200 | 1024 à 1536 |    1024    | 3072 à 3584 |    17280   |
|    Facteur de blockage |   max 5:1  |   max 2:1   |   max 8:1  |  max 4.7:1  |   max 2:1  |
| Granularité des tâches | /proc /GPU |  /proc /GPU | /proc /GPU |  /proc /GPU |   /noeud   |
|         Durée maximale |   7 jours  |   28 jours  |   7 jours  |   7 jours   |   1 jour   |

* Stockage : le tout sera décrit au dernier chapitre.

## Points à retenir
* Prévoir les **paramètres d'une tâche Slurm**
  * Nombre de processeurs (CPU) et de noeuds de calcul
  * Nombre d'accélérateurs (GPU)
  * Quantité de mémoire-vive (RAM)
  * Temps du calcul (`JJ-H:M` ou `H:M:S`)
* Différents **outils pour surveiller** les ressources utilisées
  * `time` et autres bibliothèques de mesure du temps écoulé
  * `top`, `htop`, `nvtop`, `nvidia-smi`
  * `sacct`, `seff`
  * `du -bs`, `find | wc -l` et autres outils du système d'exploitation
* On vise une **efficacité de 90%** et plus pour les tâches CPU
  * L'accélération avec un accélérateur (GPU) doit être significative (>5x)
* Le choix de la grappe dépend des besoins de chaque type de calcul